In [ ]:
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, Reshape, UpSampling2D, BatchNormalization, AveragePooling2D, Activation, Lambda, Conv2DTranspose, ReLU
from keras.models import Model, Sequential, load_model
from keras.regularizers import l1

in_height = 720//2
in_width =  1280//2

#out_height = 720
#out_width = 1280

out_height = in_height
out_width = in_width

In [ ]:
from keras.constraints import maxnorm
from keras.losses import *
encoder_input =  Input(shape=(in_height,in_width,3))

# Encoder Layers
encoder = Conv2D(8, (3, 3),padding='same', kernel_constraint=maxnorm(3))(encoder_input)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same',)(encoder)
encoder = Conv2D(16, (3, 3), padding='same', kernel_constraint=maxnorm(3))(encoder)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same')(encoder)

encoder = Conv2D(16, (3, 3), padding='same', kernel_constraint=maxnorm(3))(encoder)
encoder = Activation('relu')(encoder)
encoder = AveragePooling2D((2, 2), padding='same')(encoder)

encoder =  Flatten(name='flatten_1')(encoder)

encoder_model = Model(encoder_input,encoder)
encoder_model.summary()

encoder_model.load_weights("16C-360x640-encoder.h5")

# Decoder Layers
decoder_input = Input(shape=(45 * 80 * 16,))
decoder = Reshape(( 45, 80, 16))(decoder_input)
decoder = Conv2DTranspose(128, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(3))(decoder)
decoder = Activation('relu')(decoder)

decoder = Conv2DTranspose(64, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(3))(decoder)
decoder = Activation('relu')(decoder)

decoder = Conv2DTranspose(32, (3, 3), padding='same',strides=2, kernel_constraint=maxnorm(3))(decoder)

decoder = Conv2DTranspose(3, (1, 1), padding='same')(decoder)

#decoder = ReLU(max_value=255)(decoder)
decoder_model = Model(decoder_input,decoder)
decoder_model.summary()

autoencoder = Model(encoder_input,decoder_model(encoder))
autoencoder.compile(optimizer='adam',  loss=logcosh,metrics=['acc'])

autoencoder.load_weights('16C-Text--Images-autoencoder-weights-model-13-360x640-360x640.h5')

In [ ]:
autoencoder = load_model("./autoencoder-model-11-360x640-720x1280.h5")
autoencoder.compile(optimizer='adam', loss='mean_absolute_error',metrics=['acc'])

encoder_model = Model(autoencoder.input,autoencoder.layers[7].output)

decoder_input = Input(shape=autoencoder.layers[8].input_shape[1:])
decoder = decoder_input
for layer in autoencoder.layers[8:]:
    decoder = layer(decoder)
    
decoder_model = Model(decoder_input,decoder)

autoencoder.summary()

In [ ]:
tfjs.converters.save_keras_model(autoencoder, './tensorflowjs-model/')

In [ ]:
import cv2, os
import numpy as np
#import pandas as pd

train_dir = "../Datasets/DIV2K_train_HR/"
valid_dir = "../Datasets/DIV2K_valid_HR/"

X = list()
Y = list()
#files = pd.read_csv('image_data.csv').Path
train_files = os.listdir(train_dir)
valid_files = os.listdir(valid_dir)

def getImage(file):
    #‪C:\Users\mayan\Downloads\2-dog.jpg
    frame = cv2.imread(file,1)
    return [np.array([cv2.resize(frame,(in_width,in_height))]),np.array([cv2.resize(frame,(out_width,out_height))])]

def getValidation():
    x_valid,y_valid = [],[]   
    for file in valid_files:
        frame = cv2.imread(valid_dir + file,1)
        try:
            #y_valid.append(cv2.resize(frame,(out_width,out_height)))
            x_valid.append(cv2.resize(frame,(in_width,in_height)))
        except Exception as ex:
            print(ex)
    return np.array(x_valid),np.array(y_valid)


for file in train_files:
    frame = cv2.imread(train_dir + file,1)
    try:
        X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)

"""
train_dir = "../Datasets/Text Images/"
train_files = os.listdir(train_dir)

for file in train_files:
    frame = cv2.imread(train_dir + file,1)
    try:
        X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)
        
x_valid,y_valid = getValidation()
"""

In [ ]:
train_files = os.listdir("./../Datasets/flickr30k_images/")
for file in train_files:
    frame = cv2.imread( './../Datasets/flickr30k_images/'+ file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)
    
X = np.array(X)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)

In [ ]:
from IPython.display import clear_output
from keras.optimizers import Adam, Adadelta

validation_accuracy = 0.7050989866256714
learning_rate = (1-validation_accuracy)/10000
#autoencoder.compile(optimizer=Adam(lr = learning_rate),  loss='mean_absolute_error',metrics=['acc'])

#autoencoder.compile(optimizer=Adam(lr = 2.9490101951523684e-05),  loss='mse',metrics=['acc'])
it = datagen.flow(np.array(X),batch_size=512,shuffle = True)

x_test = getImage(train_dir+'IMG_20191018_232049.jpg')[0];

for i in range(1000):
    X_ = next(it)        
    hist = autoencoder.fit(X_, X_,
                epochs=10,
                batch_size=8,validation_data = (x_valid,x_valid))
    clear_output() 
    
    decoded_imgs = autoencoder.predict(x_test)
    h = autoencoder.evaluate(x_test,x_test)
        
    print("Validation Loss",np.mean(hist.history['val_loss']))
    print("Validation Accuracy",np.mean(hist.history['val_acc']))   
    
    print("Test Loss",h[0])
    print("Test Accuracy",h[1])
    
    cv2.imwrite("./restructured/test_image.jpg" ,decoded_imgs[0])
    cv2.imwrite("./restructured/original-test_image.jpg" ,x_test[0])
    
    decoder_model.save("16C-%dx%d-decoder.h5"%(out_height,out_width))
    encoder_model.save("16C-%dx%d-encoder.h5"%(in_height,in_width))
    '''
    encoder_model.save_weights('encoder-weights-%dx%d.h5'%(in_height,in_width)) 
    '''
    autoencoder.save_weights('16C-Text--Images-autoencoder-weights-model-13-360x640-360x640.h5')


In [ ]:
it = datagen.flow(x_test,batch_size=1)

In [ ]:
decoded_imgs = next(it)
cv2.imshow("Reconstructed", decoded_imgs[0]/255) 


cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
x_test = getImage('C:/Users/mayan/Desktop/tensorflow-autoencoder/static/pexels-photo-257840.jpeg')[0];
#x_test = getImage(train_dir + 'IMG_20191018_232049.jpg')[0]
#x_test = x_valid[4:5]

#decoded_imgs = autoencoder.predict(x_test)
import sys

decoded_imgs = encoder_model.predict(x_test)

decoded_imgs = decoded_imgs/np.max(decoded_imgs) * 255
decoded_imgs  = decoded_imgs.astype('uint8')

decoded_imgs = decoder_model.predict(decoded_imgs)
decoded_imgs = decoded_imgs/np.max(decoded_imgs)

h = autoencoder.evaluate(x_test,x_test)

print("Test Loss",h[0])
print("Test Accuracy",h[1])


cv2.imshow("Reconstructed", decoded_imgs[0]) 
cv2.imshow("Original", x_test[0]) 

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
encoder_model.save("encoder-final-model.h5")
decoder_model.save("decoder-final-model.h5")

In [ ]:
train_files = os.listdir("C:/Users/mayan/Machine Learning/Datasets/Events/")
for file in train_files:
    frame = cv2.imread("C:/Users/mayan/Machine Learning/Datasets/Events/" + file,1)
    try:
        height, width, channels = frame.shape
        if height >= in_height and width >= in_width:
            X.append(cv2.resize(frame,(in_width,in_height)))
    except Exception as ex:
        print(ex)

In [ ]:
h = autoencoder.evaluate(x_test,x_test)
#autoencoder.metrics_names
h

In [ ]:
"""
{'learning_rate': 2.9490101951523684e-05,
 'beta_1': 0.8999999761581421,
 'beta_2': 0.9990000128746033,
 'decay': 0.0,
 'epsilon': 1e-07,
 'amsgrad': False}
"""

autoencoder.optimizer.get_config()

In [ ]:
##Save Model
json= decoder_model.to_json()
with open("decoder.json", "w") as json_file:
    json_file.write(json)

decoder_model.save_weights("decoder.h5")

json= encoder_model.to_json()
with open("encoder.json", "w") as json_file:
    json_file.write(json)
encoder_model.save_weights("encoder.h5")

print("Saved model to disk")

In [ ]:
#x_test,y_test = next(getData(16))
x_valid,y_valid = getImage('C:/Users/mayan/Downloads/2-dog.jpg')
#x_test,y_test = getImage('dark-fantasy-wallpapers-28123-6689698.jpg')


In [ ]:
import matplotlib.pyplot as plt
import imutils, time

%matplotlib inline

num_images = 10
np.random.seed(42)

random_test_images = np.random.randint(x_valid.shape[0], size=min(num_images,x_valid.size))

encoded_imgs = encoder_model.predict(x_valid)
decoded_imgs = decoder_model.predict(encoded_imgs)

#decoded_imgs = autoencoder.predict(x_test)

plt.figure(figsize=(60, 50))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(3, num_images, i + 1)
    plt.imshow(x_valid[image_idx])
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    cv2.imwrite("./real/%s.jpg" %time.time(),x_valid[image_idx])
    '''
    # plot encoded image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    plt.imshow(encoded_imgs[image_idx].reshape(32, 16))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    '''
    # plot reconstructed image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    name = time.time()
    cv2.imwrite("./restructured/%s.jpg" %name,decoded_imgs[image_idx] )
    cv2.imwrite("./restructured/original-%s.jpg" %name,x_valid[image_idx] )
    plt.imshow(decoded_imgs[image_idx]/255)
    #plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()